<a href="https://colab.research.google.com/github/Flipeviotto/a_little_is_enough_attack/blob/Version-3/Ataque_LIE_artigo_VERSAO_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Subset, DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor, Normalize, Compose
from sklearn.metrics import precision_recall_fscore_support
import random

maior = 0
soma_acuracia = 0

# Classe Arguments para encapsular hiperparâmetros
class Arguments:
    def __init__(self):
        self.train_batch_size = 83  # Conforme o artigo
        self.test_batch_size = 83 # Tamanho do lote de teste maior para avaliação mais eficiente
        self.epochs = 150
        self.learning_rate = 0.1
        self.momentum = 0.9
        self.weight_decay = 1e-4
        self.n_workers = 51
        self.n_corrupted_workers = 12 # 24% dos trabalhadores são corrompidos (12/51)
        self.no_cuda = False

args = Arguments()
use_cuda = not args.no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# Definindo a rede neural (corrigido: fc1 e fc2 renomeados para hidden_layer e output_layer)
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.hidden_layer = nn.Linear(784, 100)  # 784 features de entrada, 100 neurônios ocultos
        self.output_layer = nn.Linear(100, 10)   # 100 neurônios ocultos, 10 classes de saída

    def forward(self, x):
        x = x.view(-1, 784)  # Achatar a entrada
        x = F.relu(self.hidden_layer(x))  # Aplicar ReLU na camada oculta
        x = self.output_layer(x)          # Camada de saída (sem softmax aqui, aplicado na função de perda)
        x = F.softmax(x, dim=1)           # usa softmax aqui para
        return x

# Função para preparar datasets divididos para os trabalhadores (sem mudanças necessárias)
def prepare_dataset():
    transform = Compose([
        ToTensor(),
        Normalize((0.1307,), (0.3081,))
    ])

    train_dataset = MNIST(root='./data', train=True, transform=transform, download=True)
    test_dataset = MNIST(root='./data', train=False, transform=transform)

    trainloaders = []
    subset_size = len(train_dataset) // args.n_workers
    for i in range(args.n_workers):
        worker_subset = Subset(train_dataset, range(i * subset_size, (i + 1) * subset_size))
        trainloader = DataLoader(worker_subset, batch_size=args.train_batch_size, shuffle=True)
        trainloaders.append(trainloader)

    testloader = DataLoader(test_dataset, batch_size=args.test_batch_size, shuffle=False) # Lote maior para teste

    return trainloaders, testloader

# Função de treinamento local para cada trabalhador
def train(args: Arguments, models: list, device, train_loaders, optimizers, loss_fns, epoch):
    loss = [0] * args.n_workers

    for j in range(args.n_workers):
        models[j].train()
        for data, target in train_loaders[j]:
            data, target = data.to(device), target.to(device)
            optimizers[j].zero_grad()
            output = models[j](data)
            loss[j] = loss_fns[j](output, target)

            loss[j].backward()

            # Clipping para evitar explosão de gradientes
            torch.nn.utils.clip_grad_norm_(models[j].parameters(), max_norm=1.0)
            optimizers[j].step()

# Função de agregação FedAvg
def fedavg_aggregation(models: list, global_model):
    all_params = [list(model.parameters()) for model in models]
    average_params = [torch.stack(params).mean(0) for params in zip(*all_params)]

    for global_param, avg_param in zip(global_model.parameters(), average_params):
        global_param.data.copy_(avg_param)

    # Atualiza os modelos locais com os pesos do modelo global
    for model in models:
        model.load_state_dict(global_model.state_dict())

# Função de teste para o modelo global
def test(global_model, device, test_loader, epoch):
    global maior
    global soma_acuracia
    global_model.eval()
    all_predictions = []
    all_targets = []
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = global_model(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            all_predictions.extend(pred.cpu().numpy().flatten())
            all_targets.extend(target.cpu().numpy().flatten())

    total_samples = len(test_loader.dataset)
    precision, recall, f1_score, _ = precision_recall_fscore_support(all_targets, all_predictions, average='weighted', zero_division=1)
    accuracy = (correct / total_samples) * 100
    soma_acuracia = soma_acuracia + accuracy
    if(accuracy>maior):
      maior = accuracy
      print(f"Maior acurácia: {maior}")
    with open(f"teste{wm}.txt", 'a') as file:
      file.write(f'epoca {epoch} | Accuracy: {accuracy:.2f}% | Correct: {correct}/{total_samples} | Precision: {precision:.4f} | Recall: {recall:.4f} | F1-score: {f1_score:.4f}\n')
    #print(f'Teste: Precisão: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1_score:.4f}, Acurácia: {accuracy:.2f}%')


from scipy.stats import norm

def ataque(args:Arguments, model:list, workers_maliciosos:list):
    total_clientes = args.n_workers

    #calcular a taxa de perturbação
    #s = total_clientes/2 + 1 - len(workers_maliciosos)
    #possibilidade = (total_clientes-s)/total_clientes
    #z = norm.ppf(1 - s / args.n_workers)

    #s = (args.n_workers // 2 + 1) - args.n_corrupted_workers
    #z = norm.ppf(1 - s / args.n_workers)

    z = 1.5

    # somar parametros
    soma_parametros = MLP().to(device)
    media = MLP().to(device)
    desvio_padrao = MLP().to(device)
    desvio_padrao_aux = MLP().to(device)

    # inicializa programas com zero
    for parametro, parametro_media, desvio, desvio_pad in zip(soma_parametros.parameters(), media.parameters(), desvio_padrao_aux.parameters(), desvio_padrao.parameters()):
        desvio_pad.data.zero_()
        parametro.data.zero_()
        parametro_media.data.zero_()
        desvio.data.zero_()

    with torch.no_grad():
      for malicioso in workers_maliciosos:
        for modelo_param, soma_param in zip(model[malicioso].parameters(), soma_parametros.parameters()):
          soma_param.data += modelo_param.data

      #calcular media
      for soma_param, parametro_media in zip(soma_parametros.parameters(), media.parameters()):
        parametro_media.data = soma_param.data/len(workers_maliciosos)

      # calculo desvio padrao
      for malicioso in workers_maliciosos:
        for parametro_modelo, variancia, parametro_media in zip(model[malicioso].parameters(), desvio_padrao_aux.parameters(), media.parameters()):
            diff_squared = (parametro_modelo.data - parametro_media.data) ** 2
            variancia.data += diff_squared  # Acumula diferença ao quadrado

      for variancia, desvio in zip(desvio_padrao_aux.parameters(), desvio_padrao.parameters()):
        desvio.data = torch.sqrt(variancia.data / len(workers_maliciosos))

      # alterar parametros
      for malicioso in workers_maliciosos:
        for parametro, desvio, parametro_media in zip(model[malicioso].parameters(), desvio_padrao.parameters(), media.parameters()):
          parametro.data = parametro_media.data + z * desvio.data

    return 0




# Função principal
def main(wm):
    models = [MLP().to(device) for _ in range(args.n_workers)]
    optimizers = [optim.SGD(model.parameters(), lr=args.learning_rate, momentum=args.momentum, weight_decay=args.weight_decay) for model in models]
    loss_fns = [F.cross_entropy for _ in range(args.n_workers)]

    workers_maliciosos = []
    #workers_maliciosos = random.sample(range(args.n_workers), args.n_corrupted_workers)
    for i in range(0,wm):
       workers_maliciosos.append(i)

    # Criando os loaders de dados para cada trabalhador e o loader de teste global
    trainloaders, testloader = prepare_dataset()
    global_model = MLP().to(device)

    # Loop de treinamento federado
    for epoch in range(1, args.epochs + 1):
        train(args, models, device, trainloaders, optimizers, loss_fns, epoch)



        if epoch % 1 == 0:
            if wm>1:
                ataque(args, models, workers_maliciosos)
            fedavg_aggregation(models, global_model)
            test(global_model, device, testloader, epoch)

if __name__ == "__main__":
  for wm in [args.n_workers]:                  # passo com as quantidades de workers maliciosos em cada simulação
    print(f"Quantidade de workers maliciosos: {wm}")
    with open(f"teste{wm}.txt", 'w') as file:
      file.write(f"Quantidade de workers maliciosos: {wm}\n")
    main(wm)
    print(f"media_acuracia: {soma_acuracia/150}")
    print(f"Maior acurácia: {maior}")
    with open(f"teste51.txt", 'a') as file:
      file.write(f"Maior acurácia: {maior}\n")



Quantidade de workers maliciosos: 51
Maior acurácia: 13.29
Maior acurácia: 26.490000000000002
Maior acurácia: 52.949999999999996
Maior acurácia: 72.36
Maior acurácia: 76.4
Maior acurácia: 79.86
Maior acurácia: 81.41000000000001
Maior acurácia: 84.17999999999999
Maior acurácia: 84.33
Maior acurácia: 86.14
Maior acurácia: 86.16
Maior acurácia: 86.46000000000001
Maior acurácia: 86.61
Maior acurácia: 87.4
Maior acurácia: 88.27000000000001
Maior acurácia: 88.3
Maior acurácia: 88.42
Maior acurácia: 88.51
Maior acurácia: 88.8
Maior acurácia: 88.96
Maior acurácia: 89.12
Maior acurácia: 89.21
Maior acurácia: 89.44
Maior acurácia: 89.64
Maior acurácia: 89.73


KeyboardInterrupt: 